<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/getScreenerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import openpyxl

from google.colab import drive
drive.mount('/content/drive')

from google.colab import data_table

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Try-Hyperlink.xlsx')


def _get_link_if_exists(cell) -> str | None:
    try:
        return cell.hyperlink.target
    except AttributeError:
        return None


def extract_hyperlinks_from_xlsx(
    file_name: str, sheet_name: str, columns_to_parse: list[str], row_header: int = 1
) -> pd.DataFrame:
    df = pd.read_excel(file_name, sheet_name)
    ws = openpyxl.load_workbook(file_name)[sheet_name]
    for column in columns_to_parse:
        row_offset = row_header + 1
        column_index = list(df.columns).index(column) + 1
        df[column] = [
            _get_link_if_exists(ws.cell(row=row_offset + i, column=column_index))
            for i in range(len(df[column]))
        ]
    return df

df_data = extract_hyperlinks_from_xlsx('/content/drive/My Drive/tradevesting/data/myScreenerDB.xlsx','data', ['Name'])


Mounted at /content/drive


In [ ]:
df_data

,Name,CMP Rs.,ATH%,P/E,EPS 12M Rs.,CMP / BV,Mar Cap Rs.Cr.,ROCE %,ROE %,5Yrs PE,ROCE 5Yr %,ROE 5Yr %,Debt / Eq,Sales Var 5Yrs %,Profit Var 5Yrs %,OPM %,NP 12M Rs.Cr.,CapType
0,https://www.screener.in/company/RELIANCE/conso...,1292.20,24.51,25.75,50.19,2.13,1748652.74,9.61,9.25,26.42,9.01,8.73,0.44,9.61,11.98,17.41,77652.00,LC
1,https://www.screener.in/company/TCS/consolidated/,4270.85,7.53,32.09,131.12,15.22,1545230.91,64.28,51.51,30.91,54.63,43.77,0.09,10.46,8.21,26.92,47659.00,LC
2,https://www.screener.in/company/HDFCBANK/conso...,1796.05,2.23,19.84,90.94,2.99,1372709.24,7.67,17.14,20.45,6.67,16.85,6.81,21.95,23.42,33.16,71546.36,LC
3,https://www.screener.in/company/BHARTIARTL/con...,1627.15,9.33,68.70,21.63,10.64,974249.39,13.13,14.93,65.48,9.59,12.36,2.59,13.17,122.47,51.93,13815.50,LC
4,https://www.screener.in/company/ICICIBANK/cons...,1300.10,4.79,19.37,67.37,3.56,917336.17,7.60,18.80,21.70,6.20,15.23,6.45,17.25,59.73,32.82,50084.00,LC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,https://www.screener.in/company/5PAISA/,491.65,54.40,25.50,19.29,2.66,1535.12,12.89,10.29,40.89,9.46,8.32,0.29,45.42,36.50,29.63,60.19,SC
532,https://www.screener.in/company/SURYODAY/,139.79,112.00,6.36,21.99,0.82,1485.76,8.49,12.74,10.99,7.83,4.56,5.66,24.55,19.02,44.10,233.51,SC
533,https://www.screener.in/company/ASIANTILES/con...,69.32,463.72,NaN,-0.02,0.69,965.23,1.23,-0.91,24.98,5.57,1.80,0.17,5.22,NaN,3.42,-8.55,SC
534,https://www.screener.in/company/VALIANTORG/con...,337.35,490.19,NaN,-15.03,1.30,931.05,1.07,-0.91,19.22,21.02,16.03,0.28,0.88,NaN,2.58,-41.17,SC


In [ ]:
df_data.columns

Index(['Name', 'CMP Rs.', 'ATH%', 'P/E', 'EPS 12M Rs.', 'CMP / BV',
       'Mar Cap Rs.Cr. ', 'ROCE %', 'ROE %', '5Yrs PE', 'ROCE 5Yr %',
       'ROE 5Yr %', 'Debt / Eq', 'Sales Var 5Yrs %', 'Profit Var 5Yrs %',
       'OPM %', 'NP 12M Rs.Cr.', 'CapType'],
      dtype='object')

In [ ]:
df_data['Symbol'] = df_data['Name'].str.split('company/').str[1].str.split('/').str[0]
df_data.columns = ['Name', 'CMP', 'ATH%', 'PE', 'EPS', 'PB', 'MCap', 'ROCE%', 'ROE%', 'MedPE', 'ROCE_5Yr%',
       'ROE_5Yr%', 'Debt2EqR', 'SalesVar_5Yrs%', 'ProfitVar_5Yr%', 'OPM%', 'PAT_12M', 'CapType', 'Symbol']

In [ ]:
df_data.to_csv('/content/drive/My Drive/tradevesting/data/myScreenerDB.csv', index=False)

In [ ]:
cols = ['Symbol','PE', 'EPS','MCap','ROCE%','ROE%','MedPE','ROCE_5Yr%','ROE_5Yr%','Debt2EqR','SalesVar_5Yrs%','ProfitVar_5Yr%','OPM%','PAT_12M','PB','CapType']
df = df_data[cols]

In [ ]:
df_tmp = df[(df['PE'] < df['MedPE']) & (df['Debt2EqR'] < .25) & (df['SalesVar_5Yrs%'] > 15) & (df['ProfitVar_5Yr%'] > 25)]
data_table.DataTable(df_tmp, include_index=False)

,Symbol,PE,EPS,MCap,ROCE%,ROE%,MedPE,ROCE_5Yr%,ROE_5Yr%,Debt2EqR,SalesVar_5Yrs%,ProfitVar_5Yr%,OPM%,PAT_12M,PB,CapType
89,MANKIND,51.20,52.15,102617.59,24.57,19.71,54.43,29.02,21.53,0.02,19.33,27.24,26.30,2089.16,9.51,LC
102,TATACONSUM,69.41,11.70,94854.45,10.65,8.32,72.07,9.20,6.83,0.15,15.96,26.95,15.31,1170.23,4.96,MC
146,IRCTC,54.27,15.00,65276.00,53.77,40.45,69.11,48.45,37.51,0.01,17.96,32.20,33.77,1199.87,18.53,MC
151,PIIND,34.75,117.00,61681.00,24.00,21.13,47.11,21.60,18.48,0.02,21.96,33.10,27.83,1775.10,6.46,MC
226,KPITTECH,51.68,26.54,37524.80,38.36,31.19,57.77,25.03,22.86,0.14,50.01,49.45,20.62,730.58,15.01,MC
266,MEDANTA,59.18,18.18,28877.17,19.32,17.93,61.22,12.21,12.01,0.22,17.60,56.23,23.39,487.97,9.19,SC
275,STARHEALTH,32.02,14.72,27600.99,17.69,14.29,43.84,-5.00,-2.05,0.07,30.01,45.81,7.56,862.09,3.93,SC
339,BIKAJI,67.48,11.90,20098.18,29.65,24.73,75.23,18.72,16.85,0.13,20.56,39.54,17.59,297.83,15.03,SC
386,TBOTEK,78.95,20.55,17228.54,42.93,46.05,90.92,31.25,31.84,0.20,25.87,45.96,18.10,218.21,16.01,SC
409,RRKABEL,61.86,23.31,16271.68,21.71,20.75,64.64,17.05,22.94,0.24,22.53,27.92,5.83,263.04,8.11,SC
